In [1]:
%autosave 0

Autosave disabled


# 主题提取
- TF-IDF
- TextRank
- LDA

In [1]:
import pandas as pd
import jieba.analyse as analyse

## 关键词抽取——TF-IDF

In [2]:
df = pd.read_csv('./data/technology_news.csv', encoding='utf-8')
df = df.dropna()
lines = df.content.values.tolist()
content = "".join(lines)
print(" ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.468 seconds.
Prefix dict has been built succesfully.


用户 2016 互联网 手机 平台 人工智能 百度 2017 智能 技术 数据 360 服务 直播 产品 企业 安全 视频 移动 应用 网络 行业 游戏 机器人 电商 内容 中国 领域 通过 发展


## 关键词抽取——TextRank

In [3]:
df = pd.read_csv('./data/military_news.csv', encoding='utf-8')
df = df.dropna()
lines = df.content.values.tolist()
content = "".join(lines)
print(" ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns','n','vn','v'))))

中国 海军 训练 美国 部队 进行 官兵 航母 作战 任务 能力 军事 发展 工作 国家 问题 建设 导弹 编队 记者


## LDA主题模型
- 对每一句分词，去停用词，转换成\[[w1，w2,...],[w,...],...,[...]]格式
- dictionary
- corpus
- lda_model

In [2]:
from gensim import corpora, models, similarities
import gensim

In [3]:
stopwords = pd.read_csv('./data/stopwords.txt', index_col=False, quoting=3, sep="\t", names=['stopword'], encoding='utf-8')
stopwords = stopwords['stopword'].values

In [4]:
import jieba

In [12]:
df = pd.read_csv('./data/technology_news.csv', encoding='utf-8')
df = df.dropna()
lines = df.content.values.tolist()

sentences = []
for line in lines:
    try:
        segs = jieba.lcut(line)
        segs = filter(lambda x: len(x)>1, segs)
        segs = filter(lambda x: x not in stopwords, segs)
        sentences.append(list(segs))
    except Exception:
        print(line)
        continue

In [13]:
sentences[5]

['本次',
 '商汤',
 '带来',
 '黄仁勋',
 '展示',
 '遥相呼应',
 'SenseFace',
 '人脸',
 '布控',
 '系统',
 '千万级',
 '人员',
 '库中',
 '300ms',
 '识别',
 '瞬间',
 '锁定目标',
 '功耗',
 '十几',
 '当属',
 '人脸',
 '布控',
 '一大',
 '科技']

## 词袋模型

In [14]:
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [16]:
corpus[5]

[(21, 1),
 (39, 1),
 (61, 1),
 (68, 1),
 (78, 1),
 (82, 1),
 (91, 1),
 (92, 1),
 (103, 1),
 (104, 2),
 (105, 2),
 (124, 1),
 (129, 1),
 (130, 1),
 (131, 1),
 (132, 1),
 (133, 1),
 (134, 1),
 (135, 1),
 (136, 1),
 (137, 1),
 (138, 1)]

## LDA建模

In [18]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [19]:
print(lda.print_topic(3, topn=5))

0.032*"增长" + 0.025*"市场" + 0.022*"亿元" + 0.016*"显示" + 0.016*"公司"


In [20]:
for topic in lda.print_topics(num_topics=20, num_words=8):
    print(topic[1])

0.108*"游戏" + 0.027*"孩子" + 0.026*"软件" + 0.023*"家长" + 0.018*"手机" + 0.014*"玩家" + 0.014*"电脑" + 0.011*"Windows"
0.021*"联想" + 0.019*"苹果" + 0.018*"市场" + 0.012*"手机" + 0.011*"中国" + 0.010*"数据中心" + 0.008*"印度" + 0.007*"探讨"
0.020*"互联网" + 0.016*"服务" + 0.012*"城市" + 0.012*"中国" + 0.011*"企业" + 0.010*"共享" + 0.009*"网络" + 0.009*"提供"
0.032*"增长" + 0.025*"市场" + 0.022*"亿元" + 0.016*"显示" + 0.016*"公司" + 0.015*"超过" + 0.013*"同比" + 0.011*"数据"
0.072*"百度" + 0.044*"人工智能" + 0.019*"腾讯" + 0.017*"公司" + 0.012*"阿里" + 0.011*"互联网" + 0.011*"领域" + 0.011*"AI"
0.023*"品牌" + 0.018*"消费者" + 0.018*"用户" + 0.017*"营销" + 0.016*"京东" + 0.015*"体验" + 0.013*"消费" + 0.012*"产品"
0.025*"企业" + 0.020*"技术" + 0.016*"发展" + 0.015*"中国" + 0.014*"服务" + 0.014*"行业" + 0.013*"领域" + 0.012*"机器人"
0.091*"手机" + 0.014*"摄像头" + 0.012*"采用" + 0.011*"充电" + 0.011*"360" + 0.011*"VR" + 0.011*"系列" + 0.010*"设计"
0.054*"内容" + 0.035*"视频" + 0.020*"媒体" + 0.018*"平台" + 0.013*"创业" + 0.012*"用户" + 0.011*"行业" + 0.009*"论坛"
0.032*"攻击" + 0.027*"网络" + 0.026*"漏洞" + 0.023*"网络安全" + 0.021*"360" +